# Prediction using pkl file(model)
#### for the same we pass values of all features values as a 2d numpy array format. feature values expected are as follows:
    loan_amnt
	funded_amnt
	term
	int_rate
	installment
	emp_length
	annual_inc
	dti
	earliest_cr_line
	revol_util
	total_pymnt
	total_rec_prncp
	total_rec_int
	recoveries
	collection_recovery_fee
	last_pymnt_amnt


In [188]:
import pickle
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [189]:
test_data_df = pd.read_csv('data/test_file.csv',low_memory=False, delimiter=',')

needed_columns = ["loan_amnt","funded_amnt","term","int_rate","installment","emp_length","annual_inc","dti","revol_util","total_pymnt","total_rec_prncp","total_rec_int","recoveries","collection_recovery_fee","last_pymnt_amnt"]
loan_stat_features = test_data_df[needed_columns]
print(loan_stat_features.shape)
loan_stat_features = loan_stat_features.dropna(how='any')
print(loan_stat_features.shape)
loan_stat_features.head()

(1000, 15)
(966, 15)


,loan_amnt,funded_amnt,term,int_rate,installment,emp_length,annual_inc,dti,revol_util,total_pymnt,total_rec_prncp,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt
0,10500.0,10500.0,60 months,15.28%,251.35,10+ years,43160.0,20.07,60.4%,14522.827478,10500.0,4022.83,0.0,0.0,4499.14
2,14000.0,14000.0,60 months,10.00%,297.46,3 years,50000.0,6.22,12.3%,17558.900006,14000.0,3558.90,0.0,0.0,2177.48
3,12000.0,12000.0,36 months,12.42%,400.99,5 years,50700.0,22.49,80.8%,14435.349992,12000.0,2435.35,0.0,0.0,414.83
4,4000.0,4000.0,36 months,6.17%,122.00,2 years,50808.0,15.00,16.3%,4225.942096,4000.0,225.94,0.0,0.0,2764.65
5,12000.0,12000.0,60 months,19.04%,311.56,10+ years,94000.0,0.50,44.6%,16106.570696,12000.0,4106.57,0.0,0.0,8645.72


In [190]:
float_cols = loan_stat_features.select_dtypes(include=['float'])
loan_stat_features[float_cols.columns] = float_cols.astype(int)

def extract_term(s):
    return int(s.split()[0])

# Apply the function to the 'term' column using the .apply() method
loan_stat_features['term'] = loan_stat_features['term'].apply(extract_term)

loan_stat_features['int_rate'] = loan_stat_features['int_rate'].str.replace('%', '')
loan_stat_features['int_rate'] = loan_stat_features['int_rate'].astype(float)

loan_stat_features['emp_length'] = loan_stat_features['emp_length'].astype(str)
loan_stat_features['emp_length'] = loan_stat_features['emp_length'].str.replace(r'\D+', '')
loan_stat_features['emp_length'] = loan_stat_features['emp_length'].astype(int)

loan_stat_features['revol_util'] = loan_stat_features['revol_util'].str.replace('%', '')
loan_stat_features['revol_util'] = loan_stat_features['revol_util'].astype(float)

loan_stat_features.head()

/var/folders/kr/h47bctjj5fsd2wbz5k1p6yww0000gn/T/ipykernel_26592/476331397.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  loan_stat_features['emp_length'] = loan_stat_features['emp_length'].str.replace(r'\D+', '')


,loan_amnt,funded_amnt,term,int_rate,installment,emp_length,annual_inc,dti,revol_util,total_pymnt,total_rec_prncp,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt
0,10500,10500,60,15.28,251,10,43160,20,60.4,14522,10500,4022,0,0,4499
2,14000,14000,60,10.00,297,3,50000,6,12.3,17558,14000,3558,0,0,2177
3,12000,12000,36,12.42,400,5,50700,22,80.8,14435,12000,2435,0,0,414
4,4000,4000,36,6.17,122,2,50808,15,16.3,4225,4000,225,0,0,2764
5,12000,12000,60,19.04,311,10,94000,0,44.6,16106,12000,4106,0,0,8645


In [191]:
test_array_un = loan_stat_features.values.tolist()
test_array = [[float(val) for val in row] for row in test_array_un]
# print(test_array)

In [192]:
# load the saved model
with open('models/model_random_forest.pkl', 'rb') as f:
    rf_model = pickle.load(f)

prob_default_rf = rf_model.predict_proba(test_array)
# print(f"Probability RF: {prob_default_rf}")
# make predictions on new data
y_pred_rf = rf_model.predict(test_array)
# print(f"Prediction RF: {y_pred_rf}")
validate_test_data_df=loan_stat_features
validate_test_data_df["loan_status_altered"] = test_data_df["loan_status"].map({"Fully Paid": 0, "Charged Off": 1, "Does not meet the credit policy. Status:Fully Paid": 0, "Does not meet the credit policy. Status:Charged Off":1})
validate_test_data_df['prediction_rf'] = y_pred_rf
validate_test_data_df['probability_rf'] = prob_default_rf[:,1]

In [193]:
with open('models/model_svm.pkl', 'rb') as f:
    svm_model = pickle.load(f)

svm_model.probability = True

prob_default_svm = svm_model.predict_proba(test_array)
# print(f"Probability SVM: {prob_default_svm}")

# make predictions on new data
y_pred_svm = svm_model.predict(test_array)
validate_test_data_df['prediction_svm'] = y_pred_svm
prob_df_list = [round(float(row[1]),5) for row in prob_default_svm]
validate_test_data_df['probability_svm'] = prob_df_list

In [194]:
display(validate_test_data_df[['loan_status_altered', 'prediction_rf','probability_rf', 'prediction_svm', 'probability_svm']])

,loan_status_altered,prediction_rf,probability_rf,prediction_svm,probability_svm
0,0,0,0.00,0,0.00181
2,0,0,0.00,0,0.00160
3,0,0,0.00,0,0.00172
4,0,0,0.00,0,0.00186
5,0,0,0.00,0,0.00163
6,0,0,0.00,0,0.00174
7,0,0,0.00,0,0.00167
8,0,0,0.00,0,0.00173
9,0,0,0.00,0,0.00192
10,0,0,0.01,0,0.00123
